In [2]:
!pip install pydub

In [ ]:
import openai
import os
import csv
import librosa
import numpy as np
from pydub import AudioSegment

def transcribe_audio(api_key, audio_file):
    client = openai.OpenAI(api_key=api_key)

    with open(audio_file, "rb") as file:
        response = client.audio.transcriptions.create(
            model="whisper-1",
            file=file
        )

    return response.text  # Extract the transcribed text

def extract_pitch(audio_file):
    """Extracts and returns the average pitch level using librosa."""
    y, sr = librosa.load(audio_file, sr=None)
    pitches, magnitudes = librosa.piptrack(y=y, sr=sr)

    # Extract pitch values
    pitch_values = []
    for i in range(pitches.shape[1]):
        index = np.argmax(magnitudes[:, i])  # Find the strongest pitch
        pitch = pitches[index, i]
        if pitch > 0:  # Ignore zero values (unvoiced frames)
            pitch_values.append(pitch)

    avg_pitch = np.mean(pitch_values) if pitch_values else 0
    return avg_pitch

def save_to_csv(results, output_file="/content/output_folder/output.csv"):
    """Save average pitch values to a CSV file."""
    os.makedirs("/content/output_folder", exist_ok=True)

    with open(output_file, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["File Name", "Average Pitch (Hz)"])
        writer.writerows(results)

def process_multiple_files(api_key, audio_files, output_file):
    results = []

    for audio_file in audio_files:
        print(f"Processing {audio_file}...")
        transcribe_audio(api_key, audio_file)  # Using Whisper API
        avg_pitch = extract_pitch(audio_file)
        results.append([os.path.basename(audio_file), avg_pitch])

    save_to_csv(results, output_file)
    print(f"All data saved to {output_file}")

if __name__ == "__main__":
    API_KEY = ""  # Replace with your OpenAI API key
    AUDIO_FILES = [
        "/content/ABB_segment_2 (1).mp3",
        "/content/Adani Wilmar Limited (NSEI_AWL) Jan-31-2024 - Audio_segment_2.mp3",
        "/content/Adani Wilmar Limited (NSEI_AWL) Jul-30-2024 - Audio_segment_2.mp3"
    ]  # Replace with your list of audio files
    OUTPUT_FILE = "/content/output_folder/output_pitchlevel.csv"

    process_multiple_files(API_KEY, AUDIO_FILES, OUTPUT_FILE)


Processing /content/ABB_segment_2 (1).mp3...
Processing /content/Adani Wilmar Limited (NSEI_AWL) Jan-31-2024 - Audio_segment_2.mp3...
Processing /content/Adani Wilmar Limited (NSEI_AWL) Jul-30-2024 - Audio_segment_2.mp3...
All data saved to /content/output_folder/output_pitchlevel.csv
